# IQB Template Notebook

This notebook demonstrates basic usage of the IQB library to:
1. Load cached network measurement data using `IQBCache`
2. Calculate IQB scores using `IQBCalculator`
3. Display and interpret the results

## Environment Setup

**Requirements:**
- Python 3.13+
- IQB workspace environment (install with `uv sync` from repository root)

**VSCode Setup:**
- Install recommended extensions (VSCode will prompt when opening the repository)
- Select kernel: `.venv/bin/python` from the repository root

**Running from command line:**
```bash
cd iqb/analysis
uv run jupyter notebook
```

## Overview

The Internet Quality Barometer (IQB) framework assesses Internet quality beyond simple speed
metrics by considering multiple use cases (gaming, video streaming, etc.) and their
specific network requirements.

This template uses pre-cached data included in the repository `./data` directory. You can
download additional data using this command:

```bash
uv run ./data/ghcache.py sync
```

This command will download larger data files stored inside GitHub releases.

In [ ]:
# Import required libraries
from iqb import IQBCache, IQBCalculator, IQBDatasetGranularity

## Step 1: Initialize IQB Components

We'll use:
- `IQBCache`: To fetch cached measurement data from the `../data` directory
- `IQBCalculator`: To compute IQB scores based on the data and configuration

In [ ]:
# Initialize cache pointing to the data directory
# The cache uses ../data relative to the repository root
cache = IQBCache(data_dir="../data")

# Initialize calculator with default IQB configuration
calculator = IQBCalculator()

## Step 2: Load Measurement Data

We'll load data for the United States from October 2024. The cache returns percentile data for all four network metrics:
- Download throughput (Mbps)
- Upload throughput (Mbps)
- Latency (ms)
- Packet loss (percentage)

In [ ]:
# Get cache entry for the [2024-10-01, 2024-11-01) month with country data
entry = cache.get_cache_entry(
    start_date="2024-10-01",
    end_date="2024-11-01",
    granularity=IQBDatasetGranularity.COUNTRY,
)

# Filter the M-Lab dataset to focus on the US
df_pair = entry.mlab.read_data_frame_pair(country_code="US")

print(df_pair.download)
print(df_pair.upload)

# Extract the 50 percentile
p50 = df_pair.to_iqb_data(percentile=50)

print(p50)

# Convert the p50 class to dict
p50_dict = p50.to_dict()

# Create data compatible with the IQB calculator
data = {"m-lab": p50_dict}

print(data)


## Step 3: Calculate IQB Score

The IQB score is calculated through a weighted aggregation process:
1. Compare measurements against quality thresholds for each use case
2. Compute binary requirement scores per dataset
3. Aggregate across datasets → requirement agreement scores
4. Aggregate across network requirements → use-case scores
5. Aggregate across use cases → final IQB score

Scores range from 0.0 to 1.0.

In [ ]:
# Calculate IQB score with detailed output
score = calculator.calculate_iqb_score(data=data)

print(f"\n{'=' * 50}")
print(f"Final IQB Score: {score:.3f}")
print(f"{'=' * 50}")

## Step 4: Examine IQB Configuration (Optional)

The IQB calculator uses predefined weights and thresholds for each use case and network requirement. Let's examine the configuration:

In [ ]:
# Display the IQB configuration (weights and thresholds)
calculator.print_config()

## Next Steps

This template demonstrates basic IQB usage. You can extend this by:

1. **Comparing countries**: Load data for Germany ("DE") or Brazil ("BR")
2. **Exploring percentiles**: Try different percentiles (e.g., 50, 75, 99) to understand sensitivity
3. **Custom configurations**: Modify weights and thresholds to explore parameter sensitivity
4. **Visualization**: Add plots to visualize metric distributions and score components

Please, **do not modify** this template. Rather, create a copy and modify that!